In [2]:
import numpy as np
from matplotlib import pyplot as plt
from tensorflow.keras.optimizers import Adam
#import tensorflow as tf
from datetime import datetime 
from keras_unet_collection import models, losses
import cv2
from sklearn.utils import shuffle
from PIL import Image
import os
from PIL import Image
import pickle
import segmentation_models as sm
from tensorflow.keras.callbacks import EarlyStopping
import os
import os 
import sys 
import random 
import pickle
import warnings
import numpy as np 
from time import time 
import segmentation_models as sm
import matplotlib.pyplot as plt 
from tqdm import tqdm
from itertools import chain
from skimage.io import imread, imshow, imread_collection, concatenate_images
from skimage.transform import resize
from skimage.morphology import label
from keras.models import load_model
import tensorflow as tf 
from tensorflow import keras
from tensorflow.keras.layers import Input, Dropout, Lambda, Conv2D, Conv2DTranspose, MaxPooling2D, concatenate, Activation, add, multiply, UpSampling2D
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras import backend as K 
smooth = 1

def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = tf.reduce_sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (tf.reduce_sum(y_true_f) + tf.reduce_sum(y_pred_f) + smooth)

def dice_coef_loss(y_true, y_pred):
    return 1.0 - dice_coef(y_true, y_pred)
from tensorflow.keras import backend as K 
smooth = 1


def iou_score(y_pred, y_true, smooth=1):
    intersection = K.sum(K.abs(y_true * y_pred), axis=-1)
    union = K.sum(y_true, -1) + K.sum(y_pred, -1) - intersection
    iou = (intersection + smooth)/(union + smooth)
    return iou
if __name__ == '__main__':
    gpu=int(input("Which gpu u want?"))
    os.environ["CUDA_VISIBLE_DEVICES"]=str(gpu)
    image_directory = '/home/pranab_2021cs25/Segmentation_data/Lung_Segmentation_Data/Lung_Segmentation_Data/Train/COVID-19/images/'
    mask_directory = '/home/pranab_2021cs25/Segmentation_data/Lung_Segmentation_Data/Lung_Segmentation_Data/Train/COVID-19/lung masks/'
    images=[]
    my_train_samples_covid_19=os.listdir(image_directory)
    my_train_samples_non_covid=os.listdir("/home/pranab_2021cs25/Segmentation_data/Lung_Segmentation_Data/Lung_Segmentation_Data/Train/Non-COVID/images/")
    my_train_samples_normal=os.listdir("/home/pranab_2021cs25/Segmentation_data/Lung_Segmentation_Data/Lung_Segmentation_Data/Train/Normal/images/")
    for i in my_train_samples_covid_19:
        images.append(image_directory+i)
    for i in my_train_samples_non_covid:
        images.append("/home/pranab_2021cs25/Segmentation_data/Lung_Segmentation_Data/Lung_Segmentation_Data/Train/Non-COVID/images/"+i)
    for i in my_train_samples_normal:
        images.append("/home/pranab_2021cs25/Segmentation_data/Lung_Segmentation_Data/Lung_Segmentation_Data/Train/Normal/images/"+i)
    print("appending masks")
    masks=[]
    my_train_samples_covid_19=os.listdir(mask_directory)
    my_train_samples_non_covid=os.listdir("/home/pranab_2021cs25/Segmentation_data/Lung_Segmentation_Data/Lung_Segmentation_Data/Train/Non-COVID/lung masks/")
    my_train_samples_normal=os.listdir("/home/pranab_2021cs25/Segmentation_data/Lung_Segmentation_Data/Lung_Segmentation_Data/Train/Normal/lung masks/")
    for i in my_train_samples_covid_19:
        masks.append(mask_directory+i)
    for i in my_train_samples_non_covid:
        masks.append("/home/pranab_2021cs25/Segmentation_data/Lung_Segmentation_Data/Lung_Segmentation_Data/Train/Non-COVID/lung masks/"+i)
    for i in my_train_samples_normal:
        masks.append("/home/pranab_2021cs25/Segmentation_data/Lung_Segmentation_Data/Lung_Segmentation_Data/Train/Normal/lung masks/"+i)
    print("shuffling images and masks")
    images=shuffle(images, random_state=0)
    masks=shuffle(masks,random_state=0)
    for i in images:
        x=cv2.imread(i)
        i=np.array(x)
        if i.shape!=(256,256,3):
            print(i.shape)
    for i in masks:
        x=cv2.imread(i)
        i=np.array(x)
        if i.shape!=(256,256,3):
            print(i.shape)
            
    print("creating image and mask dataset")
    image_dataset = []  #Many ways to handle data, you can use pandas. Here, we are using a list format.  
    mask_dataset = []  #Place holders to define add labels. We will add 0 to all parasitized images and 1 to uninfected.


    for i, image_name in enumerate(images):    #Remember enumerate method adds a counter and returns the enumerate object
        #if (image_name.split('.')[1] == 'tif'):
            #print(image_directory+image_name)

        img = Image.open(image_name).convert('L')
        image=np.array(img)
        #image = image.resize(image,(SIZE, SIZE))[:,:,0]
        image_dataset.append(image)

    #Iterate through all images in Uninfected folder, resize to 64 x 64
    #Then save into the same numpy array 'dataset' but with label 1


    for i, image_name in enumerate(masks):
        img = Image.open(image_name).convert('L')
        image=np.array(img)
        #image = image.resize(image,(SIZE, SIZE))[:,:,0]
        mask_dataset.append(image)

    #Normalize images
    print("doing expand dims")
    #mage_dataset=np.stack((image_dataset,)*3,axis=-1)
    image_dataset = np.array(image_dataset)/255.
    train_image_dataset=np.expand_dims((np.array(image_dataset)),3)
    #D not normalize masks, just rescale to 0 to 1.
    #mask_dataset=np.stack((mask_dataset,)*3,axis=-1)
    train_mask_dataset = np.expand_dims((np.array(mask_dataset)),3) /255.
    
    
    
    
    #now test data
    image_directory = '/home/pranab_2021cs25/Segmentation_data/Lung_Segmentation_Data/Lung_Segmentation_Data/Test/COVID-19/images/'
    mask_directory = '/home/pranab_2021cs25/Segmentation_data/Lung_Segmentation_Data/Lung_Segmentation_Data/Test/COVID-19/lung masks/'
    images=[]
    my_test_samples_covid_19=os.listdir(image_directory)
    my_test_samples_non_covid=os.listdir("/home/pranab_2021cs25/Segmentation_data/Lung_Segmentation_Data/Lung_Segmentation_Data/Test/Non-COVID/images/")
    my_test_samples_normal=os.listdir("/home/pranab_2021cs25/Segmentation_data/Lung_Segmentation_Data/Lung_Segmentation_Data/Test/Normal/images/")
    for i in my_test_samples_covid_19:
        images.append(image_directory+i)
    for i in my_test_samples_non_covid:
        images.append("/home/pranab_2021cs25/Segmentation_data/Lung_Segmentation_Data/Lung_Segmentation_Data/Test/Non-COVID/images/"+i)
    for i in my_test_samples_normal:
        images.append("/home/pranab_2021cs25/Segmentation_data/Lung_Segmentation_Data/Lung_Segmentation_Data/Test/Normal/images/"+i)
    print("appending masks")
    masks=[]
    my_test_samples_covid_19=os.listdir(mask_directory)
    my_test_samples_non_covid=os.listdir("/home/pranab_2021cs25/Segmentation_data/Lung_Segmentation_Data/Lung_Segmentation_Data/Test/Non-COVID/lung masks/")
    my_test_samples_normal=os.listdir("/home/pranab_2021cs25/Segmentation_data/Lung_Segmentation_Data/Lung_Segmentation_Data/Test/Normal/lung masks/")
    for i in my_test_samples_covid_19:
        masks.append(mask_directory+i)
    for i in my_test_samples_non_covid:
        masks.append("/home/pranab_2021cs25/Segmentation_data/Lung_Segmentation_Data/Lung_Segmentation_Data/Test/Non-COVID/lung masks/"+i)
    for i in my_test_samples_normal:
        masks.append("/home/pranab_2021cs25/Segmentation_data/Lung_Segmentation_Data/Lung_Segmentation_Data/Test/Normal/lung masks/"+i)
    print("shuffling images and masks")
    images=shuffle(images, random_state=0)
    masks=shuffle(masks,random_state=0)
    for i in images:
        x=cv2.imread(i)
        i=np.array(x)
        if i.shape!=(256,256,3):
            print(i.shape)
    for i in masks:
        x=cv2.imread(i)
        i=np.array(x)
        if i.shape!=(256,256,3):
            print(i.shape)
            
    print("creating image and mask dataset")
    image_dataset = []  #Many ways to handle data, you can use pandas. Here, we are using a list format.  
    mask_dataset = []  #Place holders to define add labels. We will add 0 to all parasitized images and 1 to uninfected.


    for i, image_name in enumerate(images):    #Remember enumerate method adds a counter and returns the enumerate object
        #if (image_name.split('.')[1] == 'tif'):
            #print(image_directory+image_name)

        img = Image.open(image_name).convert('L')
        image=np.array(img)
        #image = image.resize(image,(SIZE, SIZE))[:,:,0]
        image_dataset.append(image)

    #Iterate through all images in Uninfected folder, resize to 64 x 64
    #Then save into the same numpy array 'dataset' but with label 1


    for i, image_name in enumerate(masks):
        img = Image.open(image_name).convert('L')
        image=np.array(img)
        #image = image.resize(image,(SIZE, SIZE))[:,:,0]
        mask_dataset.append(image)

    #Normalize images
    print("doing expand dims")
    #mage_dataset=np.stack((image_dataset,)*3,axis=-1)
    image_dataset = np.array(image_dataset)/255.
    test_image_dataset=np.expand_dims((np.array(image_dataset)),3)
    #D not normalize masks, just rescale to 0 to 1.
    #mask_dataset=np.stack((mask_dataset,)*3,axis=-1)
    test_mask_dataset = np.expand_dims((np.array(mask_dataset)),3) /255.
    
    
    
    
    #validation data
    image_directory = '/home/pranab_2021cs25/Segmentation_data/Lung_Segmentation_Data/Lung_Segmentation_Data/Val/COVID-19/images/'
    mask_directory = '/home/pranab_2021cs25/Segmentation_data/Lung_Segmentation_Data/Lung_Segmentation_Data/Val/COVID-19/lung masks/'
    images=[]
    my_val_samples_covid_19=os.listdir(image_directory)
    my_val_samples_non_covid=os.listdir("/home/pranab_2021cs25/Segmentation_data/Lung_Segmentation_Data/Lung_Segmentation_Data/Val/Non-COVID/images/")
    my_val_samples_normal=os.listdir("/home/pranab_2021cs25/Segmentation_data/Lung_Segmentation_Data/Lung_Segmentation_Data/Val/Normal/images/")
    for i in my_val_samples_covid_19:
        images.append(image_directory+i)
    for i in my_val_samples_non_covid:
        images.append("/home/pranab_2021cs25/Segmentation_data/Lung_Segmentation_Data/Lung_Segmentation_Data/Val/Non-COVID/images/"+i)
    for i in my_val_samples_normal:
        images.append("/home/pranab_2021cs25/Segmentation_data/Lung_Segmentation_Data/Lung_Segmentation_Data/Val/Normal/images/"+i)
    print("appending masks")
    masks=[]
    my_val_samples_covid_19=os.listdir(mask_directory)
    my_val_samples_non_covid=os.listdir("/home/pranab_2021cs25/Segmentation_data/Lung_Segmentation_Data/Lung_Segmentation_Data/Val/Non-COVID/lung masks/")
    my_val_samples_normal=os.listdir("/home/pranab_2021cs25/Segmentation_data/Lung_Segmentation_Data/Lung_Segmentation_Data/Val/Normal/lung masks/")
    for i in my_val_samples_covid_19:
        masks.append(mask_directory+i)
    for i in my_val_samples_non_covid:
        masks.append("/home/pranab_2021cs25/Segmentation_data/Lung_Segmentation_Data/Lung_Segmentation_Data/Val/Non-COVID/lung masks/"+i)
    for i in my_val_samples_normal:
        masks.append("/home/pranab_2021cs25/Segmentation_data/Lung_Segmentation_Data/Lung_Segmentation_Data/Val/Normal/lung masks/"+i)
    print("shuffling images and masks")
    images=shuffle(images, random_state=0)
    masks=shuffle(masks,random_state=0)
    for i in images:
        x=cv2.imread(i)
        i=np.array(x)
        if i.shape!=(256,256,3):
            print(i.shape)
    for i in masks:
        x=cv2.imread(i)
        i=np.array(x)
        if i.shape!=(256,256,3):
            print(i.shape)
            
    print("creating image and mask dataset")
    image_dataset = []  #Many ways to handle data, you can use pandas. Here, we are using a list format.  
    mask_dataset = []  #Place holders to define add labels. We will add 0 to all parasitized images and 1 to uninfected.


    for i, image_name in enumerate(images):    #Remember enumerate method adds a counter and returns the enumerate object
        #if (image_name.split('.')[1] == 'tif'):
            #print(image_directory+image_name)

        img = Image.open(image_name).convert('L')
        image=np.array(img)
        #image = image.resize(image,(SIZE, SIZE))[:,:,0]
        image_dataset.append(image)

    #Iterate through all images in Uninfected folder, resize to 64 x 64
    #Then save into the same numpy array 'dataset' but with label 1


    for i, image_name in enumerate(masks):
        img = Image.open(image_name).convert('L')
        image=np.array(img)
        #image = image.resize(image,(SIZE, SIZE))[:,:,0]
        mask_dataset.append(image)

    #Normalize images
    print("doing expand dims")
    #mage_dataset=np.stack((image_dataset,)*3,axis=-1)
    image_dataset = np.array(image_dataset)/255.
    val_image_dataset=np.expand_dims((np.array(image_dataset)),3)
    #D not normalize masks, just rescale to 0 to 1.
    #mask_dataset=np.stack((mask_dataset,)*3,axis=-1)
    val_mask_dataset = np.expand_dims((np.array(mask_dataset)),3) /255.
    
    
    
    
    print(train_image_dataset.shape)
    print(train_mask_dataset.shape)
    print(test_image_dataset.shape)
    print(test_mask_dataset.shape)
    print(val_image_dataset.shape)
    print(val_mask_dataset.shape)
    

    
#     print("dividing data into train and test split")
#     from sklearn.model_selection import train_test_split
#     X_train, X_test, y_train, y_test = train_test_split(image_dataset, mask_dataset, test_size = 0.10, random_state = 0)
    IMG_HEIGHT = train_image_dataset.shape[1]
    IMG_WIDTH  = train_image_dataset.shape[2]
    IMG_CHANNELS = train_image_dataset.shape[3]
    num_labels = 1  #Binary
    input_shape = (IMG_HEIGHT,IMG_WIDTH,IMG_CHANNELS)
    batch_size = 32
    
    print("initialize model")
    model_r2_Unet_from_scratch = sm.Unet('vgg16', classes=1, activation='sigmoid',input_shape=(256, 256, 1),encoder_weights='imagenet')
#     model_r2_Unet_from_scratch.compile(loss=[dice_coef_loss], optimizer=Adam(lr = 1e-4), 
#               metrics=[dice_coef, 'binary_accuracy',iou_score])
    
    model_r2_Unet_from_scratch.compile(optimizer=Adam(lr=1e-4), 
              loss=[dice_coef_loss], 
           metrics = [dice_coef, 'binary_accuracy',iou_score])
    
    start5 = datetime.now() 
    early_stopping = EarlyStopping(monitor='val_loss', 
                patience=35, 

                min_delta=0.001, 
                mode='min')
    
    print("starting fitting")
    r2_Unet_from_scratch_history = model_r2_Unet_from_scratch.fit(train_image_dataset, train_mask_dataset, 
                        verbose=1,
                        batch_size = batch_size,
                        validation_data=(val_image_dataset, val_mask_dataset), 
                        shuffle=False,
                        epochs=100,callbacks=[early_stopping])
    
    history = r2_Unet_from_scratch_history
    #np.save('my_history_inceptionv3unet.npy',history.history)
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs = range(1, len(loss) + 1)
    plt.plot(epochs, loss, 'y', label='Training loss')
    plt.plot(epochs, val_loss, 'r', label='Validation loss')
    plt.title('Training and validation loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.savefig("traing and validation loss of vgg16_unet.png")
    plt.show()
    print("second plot")
    acc = history.history['iou_score']
    val_acc = history.history['val_iou_score']

    plt.plot(epochs, acc, 'y', label='Training IoU')
    plt.plot(epochs, val_acc, 'r', label='Validation IoU')
    plt.title('Training and validation IoU')
    plt.xlabel('Epochs')
    plt.ylabel('IoU')
    plt.legend()
    plt.savefig("training and validation IOU of vgg16_unet.png")
    plt.show()
    filename = 'vgg16_unet model.hdf5'
    model_r2_Unet_from_scratch.save(filename)

Which gpu u want?4


FileNotFoundError: [Errno 2] No such file or directory: '/home/pranab_2021cs25/Segmentation_data/Lung_Segmentation_Data/Lung_Segmentation_Data/Train/COVID-19/images/'